### Initializations

In [1]:
import pyodbc
import os
import json
import logging
from datetime import datetime

In [2]:
%run OEA/modules/Ed-Fi/v0.8/src/utilities/edfi_v0_8_edfi_py

In [3]:
%run EdGraph/modules/EdGraph_DW/v0.7/src/utilities/edgraph_dw_v0_7_dw_builder

In [4]:
from datetime import datetime
oea = EdFiOEAChild()   
error_logger = ErrorLogging(spark = spark,
                            oea = oea,
                            logger = logger)

In [ ]:
def assign_default_variable(variable_name, default_value):
    if variable_name not in globals():
        globals()[variable_name] = default_value
        logger.info(f'{variable_name} not found - using system default')
        

In [5]:
assign_default_variable(variable_name = 'workspace', 
                        default_value = 'sandbox2')
oea.set_workspace(workspace)

In [24]:
metadata_path = "stage3/EdGraph_DW/build-metadata/stage3-query-assets.json"
metadata_url = oea.to_url(metadata_path)

In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.types import IntegerType

### Global Query Parameters

In [ ]:
from datetime import datetime, timedelta

incremental_prefix = ''
current_datetime = datetime.now()
last_datetime = current_datetime - timedelta(days=30) # TODO: Example Offset

current_execution_datetime_str = current_datetime.strftime("%Y-%m-%dT%H:%M:%S.%f0")
last_execution_datetime_str = last_datetime.strftime("%Y-%m-%dT%H:%M:%S.%f0")

# assign_default_variable(variable_name = 'stage2_db_name', 
#                         default_value = 'ldb_sandbox2_s2r_ed_fi')

# assign_default_variable(variable_name = 'base_table_db_name', 
#                         default_value = 'ldb_sandbox2_s3_ed_fi')

# assign_default_variable(variable_name = 'stage3_db_name', 
#                         default_value = 'ldb_sandbox2_s3_ed_fi')

assign_default_variable(variable_name = 'current_school_year', 
                        default_value = 'SchoolYear')

assign_default_variable(variable_name = 'schoolYear_varParam', 
                        default_value = 'SchoolYear')

assign_default_variable(variable_name = 'districtId_varParam', 
                        default_value = 'DistrictId')

assign_default_variable(variable_name = 'present_year', #TODO: Fix
                        default_value = '2023')

### Connection Initializations

In [27]:

assign_default_variable(variable_name = 'secret_name', 
                        default_value = 'syn-oea-hisddev-ondemand-mssql-admin-credentials-password')

db_password = oea._get_secret(secret_name)

assign_default_variable(variable_name = 'server_name', 
                        default_value = 'syn-oea-hisddev-ondemand.sql.azuresynapse.net')
assign_default_variable(variable_name = 'database_name', 
                        default_value = 'sdb_sandbox2_s3')

assign_default_variable(variable_name = 'user_name', 
                        default_value = 'eduanalyticsuser')

assign_default_variable(variable_name = 'driver', 
                        default_value = 'ODBC Driver 18 for SQL Server')
password = db_password
db_password = oea._get_secret(secret_name)

### Main Code

#### Automated (via MetadataProcessor)

In [28]:
jsonDF = spark.read.option("multiline", "true").json(metadata_url).cache()

json_string = jsonDF.toJSON().collect()[0]
original_metadata = json.loads(json_string)

In [29]:
test_version = 'v3'
semantic_views_builder = SemanticViewsBuilder(original_metadata = original_metadata, 
                                              stage3_db_name = stage3_db_name,
                                              stage_3_path = f'stage3/EdGraph_DW', 
                                              partitioning = False,
                                              spark = spark, 
                                              oea = oea, 
                                              logger = logger,
                                              error_logger = error_logger,
                                              lakeTableOverwrite = True)
semantic_views_builder.process_metadata()

In [30]:
semantic_views_builder.set_server_creds(server_name = server_name, 
                                        database_name = database_name, 
                                        user_name = user_name, 
                                        password = password, 
                                        driver=driver)

In [31]:
from datetime import datetime
overwrite = True
schema_name = 'semantic'
schema_queries_in_order, schema_queries_params_in_order = semantic_views_builder.return_schema_queries_in_order(schema_name)
#table_name=['vw_AuthUser','vw_FactStudentDisciplineIncident','vw_FactStudentEarlyWarning']
table_names = list(schema_queries_in_order.keys())

semantic_views_builder.create_schema_if_not_exists(schema_name)
for table_name in table_names:
    try:
        logger.info(f'SEMANTIC VIEW CREATION - {table_name}')
        parameterized_queries = semantic_views_builder.parameterize_table_queries(
                                                schema_queries=schema_queries_in_order,
                                                schema_queries_params=schema_queries_params_in_order,
                                                table_name=table_name,
                                                stage2_db_name = stage2_db_name,
                                                stage3_db_name = stage3_db_name,
                                                base_table_db_name = base_table_db_name,
                                                current_execution_datetime_str = current_execution_datetime_str,
                                                last_execution_datetime_str = last_execution_datetime_str,
                                                current_school_year = current_school_year,
                                                schoolYear_varParam = schoolYear_varParam,
                                                districtId_varParam = districtId_varParam,
                                                incremental_prefix = incremental_prefix,
                                                # CurrentSchoolYear = current_school_year,
                                                query_params={})
        if overwrite:
            semantic_views_builder.drop_view(schema_name, table_name)    
        semantic_views_builder.execute_table_queries(schema_name,
                                                    table_name, 
                                                    parameterized_queries)
        print()
    except Exception as error:
        logger.exception(error)
        #logger.error(f"An Error Occurred while creating: {table_name}")

In [14]:
# TODO: R&D on how to implement error logging (entity-level) for semantic views
# df_logs = error_logger.create_spark_df('entity')
# error_logger.write_logs_to_delta_lake(df = df_logs, 
#                                       log_type = 'entity',
#                                      destination_url = oea.to_url(f'stage3/EdGraph_DW/etl-logs/log_type=entity/'))

In [32]:
jsonDF.unpersist()
logger.info('Cached Data Removed From Memory')

In [ ]:
semantic_views_builder.close_connection()